In [3]:
import pandas as pd
df = pd.read_csv("titanic/train.csv")

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Remove unwanted columns

In [6]:
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Fare'], axis=1) #axis is required

In [36]:
df
from numpy import nan

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Cabin     204 non-null    object 
 7   Embarked  889 non-null    object 
dtypes: float64(1), int64(4), object(3)
memory usage: 55.8+ KB


In [12]:
df.drop(['Cabin'], axis =1, inplace=True)

In [13]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


### Filling missing values in age by average age of males and females

In [46]:
df['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [51]:
df[df['Embarked'].isna()]
df.drop([61, 829], axis =0, inplace=True)

In [52]:
mean_all = df['Age'].mean()
mean_all

29.64209269662921

In [53]:
mean_male = df[df['Sex'] == 'male']['Age'].mean()
mean_male

30.72664459161148

In [54]:
mean_female = df[df['Sex'] == 'female']['Age'].mean()
mean_female

27.745173745173744

In [59]:
df[(df['Age'].isna()) & (df['Sex'] == 'male')]

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
5,0,3,male,NaN,0,0,Q
17,1,2,male,NaN,0,0,S
26,0,3,male,NaN,0,0,C
29,0,3,male,NaN,0,0,S
36,1,3,male,NaN,0,0,C
...,...,...,...,...,...,...,...
839,1,1,male,NaN,0,0,C
846,0,3,male,NaN,8,2,S
859,0,3,male,NaN,0,0,C
868,0,3,male,NaN,0,0,S


In [62]:
df[(df['Age'].isna()) & (df['Sex'] == 'male')].fillna(value=mean_male)
df[(df['Age'].isna()) & (df['Sex'] == 'male')]

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
5,0,3,male,NaN,0,0,Q
17,1,2,male,NaN,0,0,S
26,0,3,male,NaN,0,0,C
29,0,3,male,NaN,0,0,S
36,1,3,male,NaN,0,0,C
...,...,...,...,...,...,...,...
839,1,1,male,NaN,0,0,C
846,0,3,male,NaN,8,2,S
859,0,3,male,NaN,0,0,C
868,0,3,male,NaN,0,0,S
